<a href="https://colab.research.google.com/github/ken22i/LLama-Training/blob/main/LlamaTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers peft accelerate datasets bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import datasets
import torch
from peft import LoraConfig, TaskType, get_peft_model
from peft.peft_model import PeftModel
from transformers import LlamaForCausalLM as ModelCls
from transformers import Trainer, TrainingArguments

# 讀取 Model
model_name = "TheBloke/Llama-2-7B-Chat-fp16"
model: ModelCls = ModelCls.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

# 讀取 Peft Model
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)
model: PeftModel = get_peft_model(model, peft_config)
model.print_trainable_parameters()


# 讀取資料集
data_files = {
    "train": "train.tokens.json.gz",
    "dev": "dev.tokens.json.gz",
}

dataset = datasets.load_dataset(
    "json",
    data_files=data_files,
    cache_dir="cache",
)


# 設定訓練參數
output_dir = "models/Llama-7B-TwAddr-LoRA"
train_args = TrainingArguments(
    output_dir,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    eval_accumulation_steps=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-4,
    save_total_limit=3,
    num_train_epochs=5,
    load_best_model_at_end=True,
    bf16=True,
)


# 開始訓練模型
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["dev"],
)
trainer.train()

# 儲存訓練完的模型
trainer.save_model()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/635 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

Generating dev split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ken22i22i22i (ken22i22i22i-iii) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,0.188116
2,No log,0.186122
3,No log,0.183155
4,No log,0.203465
5,0.160700,0.207239


In [ ]:
from typing import Union

import torch
from peft import PeftModel
from peft.tuners.lora import LoraModel
from transformers import LlamaForCausalLM as ModelCls
from transformers import LlamaTokenizerFast as TkCls

PeftCls = Union[PeftModel, LoraModel]

orig_model = "TheBloke/Llama-2-7B-Chat-fp16"
lora_model = "models/Llama-7B-TwAddr-LoRA"
output_dir = "models/Llama-7B-TwAddr-Merged"

model = ModelCls.from_pretrained(
    orig_model,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# Due to generation config validation.
model.generation_config.temperature = 1.0
model.generation_config.top_p = 1.0

model: PeftCls = PeftModel.from_pretrained(
    model,
    lora_model,
    torch_dtype=torch.float16,
)

model = model.merge_and_unload()
model.save_pretrained(
    output_dir,
    safe_serialization=True,
)

# Tokenizer 也要跟著另外存一份
tk: TkCls = TkCls.from_pretrained(orig_model)
tk.save_pretrained(output_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/770 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

('models/Llama-7B-TwAddr-Merged/tokenizer_config.json',
 'models/Llama-7B-TwAddr-Merged/special_tokens_map.json',
 'models/Llama-7B-TwAddr-Merged/tokenizer.model',
 'models/Llama-7B-TwAddr-Merged/added_tokens.json',
 'models/Llama-7B-TwAddr-Merged/tokenizer.json')

In [ ]:
!pip install vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5

In [ ]:
import json

from vllm import LLM, SamplingParams

template = """### USER:
請將以下路名解析為 JSON 格式。
輸入：臺北市中正區八德路
輸出：{{"city": "臺北市", "town": "中正區", "road": "八德路"}}
輸入：{}
### ASSISTANT:
{}"""

template = """<|im_start|>user
請將以下路名解析為 JSON 格式。
輸入：{}
<|im_end|>
<|im_start|>assistant
{}"""

# 輸入：臺北市中正區八德路
# 輸出：{{"city": "臺北市", "town": "中正區", "road": "八德路"}}


def build_prompt(inn, out=""):
    return template.format(inn, out)


def iter_dataset(file_path):
    data = load_json(file_path)

    for item in data:
        city = item["city"]
        town = item["town"]
        road = item["road"]

        full = f"{city}{town}{road}"

        yield full, item


def load_json(file_path):
    with open(file_path, "rt", encoding="UTF-8") as fp:
        return json.load(fp)


# 建立測試集的 Prompt 列表
prompts, items = list(), list()
for full, item in iter_dataset("test.json"):
    prompt = build_prompt(full)
    prompts.append(prompt)
    items.append(item)

# 讀取模型
model_name = "models/Llama-7B-TwAddr-Merged"
llm = LLM(model_name, dtype="float16", gpu_memory_utilization=0.95, max_model_len=2048)

# temperature 設為 0.0 為 Greedy Decode
# 確保每次實驗的結果都是一樣的
sampling_params = SamplingParams(
    max_tokens=256,
    temperature=0.0,
    stop=["}"],
)



INFO 03-26 09:20:32 [__init__.py:239] Automatically detected platform cuda.
INFO 03-26 09:20:48 [config.py:585] This model supports multiple tasks: {'classify', 'generate', 'score', 'embed', 'reward'}. Defaulting to 'generate'.
WARNING 03-26 09:20:48 [arg_utils.py:1854] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 03-26 09:20:48 [llm_engine.py:241] Initializing a V0 LLM engine (v0.8.2) with config: model='models/Llama-7B-TwAddr-Merged', speculative_config=None, tokenizer='models/Llama-7B-TwAddr-Merged', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 03-26 09:22:05 [loader.py:447] Loading weights took 74.05 seconds
INFO 03-26 09:22:05 [model_runner.py:1146] Model loading took 12.5523 GB and 74.364932 seconds
INFO 03-26 09:22:08 [worker.py:267] Memory profiling takes 2.08 seconds
INFO 03-26 09:22:08 [worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.95) = 14.00GiB
INFO 03-26 09:22:08 [worker.py:267] model weights take 12.55GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.31GiB; the rest of the memory reserved for KV Cache is 1.11GiB.
INFO 03-26 09:22:08 [executor_base.py:111] # cuda blocks: 142, # CPU blocks: 512
INFO 03-26 09:22:08 [executor_base.py:116] Maximum concurrency for 2048 tokens per request: 1.11x
INFO 03-26 09:22:11 [model_runner.py:1442] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. I

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:36<00:00,  1.05s/it]

INFO 03-26 09:22:48 [model_runner.py:1570] Graph capturing finished in 37 secs, took 0.24 GiB
INFO 03-26 09:22:48 [llm_engine.py:447] init engine (profile, create kv cache, warmup model) took 42.74 seconds


In [ ]:
# 對所有 Prompt 同時進行推論
outputs = llm.generate("""<|im_start|>user
請回答以下的問題
輸入：{生成式ai是什麼}
<|im_end|>
<|im_start|>assistant
{}""", sampling_params)
# 評估生成結果
print(outputs)
# results = list()
# for out, item in zip(outputs, items):
#     text = out.outputs[0].text
#     print(text)
#     # 嘗試解析模型的輸出
#     try:
#         begin = text.index("{")
#         text = text[begin:] + "}"
#         pred = json.loads(text)
#     except:
#         pred = None

#     results.append(pred == item)
#     if pred != item:
#         print(pred, item)
#     break

# # 輸出準確率
# accuracy = sum(results) / len(results)
# print(f"Accuracy: {accuracy:.2%}")

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.51s/it, est. speed input: 18.54 toks/s, output: 12.26 toks/s]

[RequestOutput(request_id=6, prompt='<|im_start|>user\n請回答以下的問題\n輸入：{生成式ai是什麼}\n<|im_end|>\n<|im_start|>assistant\n{}', prompt_token_ids=[1, 529, 29989, 326, 29918, 2962, 29989, 29958, 1792, 13, 235, 174, 142, 30742, 234, 176, 151, 30651, 30557, 30210, 232, 152, 146, 236, 164, 143, 13, 235, 191, 187, 30752, 30383, 29912, 30486, 30494, 30607, 1794, 30392, 231, 190, 131, 236, 189, 191, 29913, 13, 29966, 29989, 326, 29918, 355, 29989, 29958, 13, 29966, 29989, 326, 29918, 2962, 29989, 29958, 465, 22137, 13, 8875], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='\n<|im_end|>\n<|im|>\n{"city": "新竹市", "town": "東勢區", "road": "環山路"', token_ids=(13, 29966, 29989, 326, 29918, 355, 29989, 29958, 13, 29966, 29989, 326, 29989, 29958, 13, 6377, 12690, 1115, 376, 30374, 31709, 30461, 613, 376, 27734, 1115, 376, 30591, 232, 142, 165, 31561, 613, 376, 9972, 1115, 376, 234, 149, 179, 30329, 30874, 9092), cumulative_logprob=None, logprobs=

In [ ]:
import json
from vllm import LLM, SamplingParams

# 測試地址輸入
test_input = "幫我把以下的地址解析為json格式，臺北市中正區忠孝東路"

# 設定取樣參數
sampling_params = SamplingParams(
    max_tokens=256,
    temperature=0.0,
    stop=["}"]
)

# 初始化模型
model_name = "models/Llama-7B-TwAddr-Merged"  # 替換為您的模型路徑或名稱
llm = LLM(model=model_name, dtype="float16", max_model_len=512)

# 生成回應
outputs = llm.generate([test_input], sampling_params)

# 解析並顯示結果
generated_text = outputs[0].outputs[0].text
print(generated_text)
# 嘗試解析模型的輸出
try:
    begin = generated_text.index("{")
    end = generated_text.index("}", begin)
    json_text = generated_text[begin:end+1]
    parsed_output = json.loads(json_text)
except Exception as e:
    parsed_output = None
    print(f"解析輸出時發生錯誤: {e}")

print("模型輸出:", parsed_output)


INFO 03-26 08:30:17 [__init__.py:239] Automatically detected platform cuda.
INFO 03-26 08:30:39 [config.py:585] This model supports multiple tasks: {'generate', 'embed', 'score', 'reward', 'classify'}. Defaulting to 'generate'.
WARNING 03-26 08:30:40 [arg_utils.py:1854] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 03-26 08:30:40 [llm_engine.py:241] Initializing a V0 LLM engine (v0.8.2) with config: model='models/Llama-7B-TwAddr-Merged', speculative_config=None, tokenizer='models/Llama-7B-TwAddr-Merged', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=512, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='x

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 03-26 08:31:49 [loader.py:447] Loading weights took 67.06 seconds
INFO 03-26 08:31:50 [model_runner.py:1146] Model loading took 12.5523 GB and 67.341454 seconds
INFO 03-26 08:31:52 [worker.py:267] Memory profiling takes 2.08 seconds
INFO 03-26 08:31:52 [worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 03-26 08:31:52 [worker.py:267] model weights take 12.55GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.31GiB; the rest of the memory reserved for KV Cache is 0.37GiB.
INFO 03-26 08:31:53 [executor_base.py:111] # cuda blocks: 47, # CPU blocks: 512
INFO 03-26 08:31:53 [executor_base.py:116] Maximum concurrency for 512 tokens per request: 1.47x
INFO 03-26 08:31:57 [model_runner.py:1442] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If 

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:36<00:00,  1.05s/it]

INFO 03-26 08:32:34 [model_runner.py:1570] Graph capturing finished in 37 secs, took 0.24 GiB
INFO 03-26 08:32:34 [llm_engine.py:447] init engine (profile, create kv cache, warmup model) took 44.18 seconds



Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.30s/it, est. speed input: 6.98 toks/s, output: 10.57 toks/s]




輸入：臺北市中正區忠孝東路
<user>
<path>
{"city": "臺北市", "town": "中正區", "road": "忠孝東路"
解析輸出時發生錯誤: substring not found
模型輸出: None
